In [ ]:
# Required packages:
# pip install pandas matplotlib seaborn sqlalchemy psycopg2-binary

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# PostgreSQL connection info (hardcoded)
PG_HOST = "64.110.115.12"
PG_DB = "exchange"
PG_USER = "exchange_admin"
PG_PASSWORD = "exchange_password"

# SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}/{PG_DB}")

# Query last 30 days
exchange_query = "SELECT * FROM exchange ORDER BY date DESC LIMIT 30"
commodities_query = "SELECT * FROM commodities ORDER BY date DESC LIMIT 30"

exchange_df = pd.read_sql(exchange_query, engine)
commodities_df = pd.read_sql(commodities_query, engine)

# Sort by date ascending
exchange_df = exchange_df.sort_values("date")
commodities_df = commodities_df.sort_values("date")

# Merge on date (inner join)
merged = pd.merge(exchange_df, commodities_df, on="date", how="inner")

# Exchange rates plot
plt.figure(figsize=(12, 6))
plt.plot(exchange_df["date"], exchange_df["usd"], label="USD")
plt.plot(exchange_df["date"], exchange_df["jpy"], label="JPY")
plt.plot(exchange_df["date"], exchange_df["eur"], label="EUR")
plt.plot(exchange_df["date"], exchange_df["cny"], label="CNY")
plt.title("Exchange Rates (Last 30 Days)")
plt.xlabel("Date")
plt.ylabel("Exchange Rate")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Commodities plot (gold, silver, copper, oil)
plt.figure(figsize=(12, 6))
plt.plot(commodities_df["date"], commodities_df["gold"], label="Gold")
plt.plot(commodities_df["date"], commodities_df["silver"], label="Silver")
plt.plot(commodities_df["date"], commodities_df["copper"], label="Copper")
plt.plot(commodities_df["date"], commodities_df["crude_oil"], label="Crude Oil")
plt.plot(commodities_df["date"], commodities_df["brent_oil"], label="Brent Oil")
plt.title("Major Commodities Prices (Last 30 Days)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Correlation analysis
corr_cols = [
    "usd", "jpy", "eur", "cny",
    "gold", "silver", "copper", "crude_oil", "brent_oil"
]
corr = merged[corr_cols].corr(method="pearson")

print("Correlation matrix:")
print(corr)

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap: Exchange Rates & Commodities")
plt.tight_layout()
plt.show()